##### Copyright 2022 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Regressão logística para classificação binária com as APIs core

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/guide/core/logistic_regression_core"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/guide/core/logistic_regression_core.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/guide/core/logistic_regression_core.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/guide/core/logistic_regression_core.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

Este guia demonstra como usar as [APIs de baixo nível do TensorFlow Core](https://www.tensorflow.org/guide/core) para realizar [classificação binária](https://developers.google.com/machine-learning/glossary#binary_classification){:.external} com [regressão logística](https://developers.google.com/machine-learning/crash-course/logistic-regression/){:.external}. Ele usa o [Wisconsin Breast Cancer Dataset](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)){:.external} para classificação de tumores de câncer de mama.

A [regressão logística](https://developers.google.com/machine-learning/crash-course/logistic-regression/){:.external} é um dos algoritmos mais populares para classificação binária. Dado um conjunto de exemplos com características, o objetivo da regressão logística é gerar valores entre 0 e 1, que podem ser interpretados como as probabilidades de cada exemplo pertencer a uma determinada classe. 

## Configuração

Este tutorial usa [pandas](https://pandas.pydata.org){:.external} para ler um arquivo CSV em um [DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html){:.external}, [seaborn](https://seaborn.pydata.org){:.external} para plotar um relacionamento de pares num dataset, [Scikit-learn](https://scikit-learn.org/){:.external} para calcular uma matriz de confusões e [matplotlib](https://matplotlib.org/){:.external} para criar visualizações.

In [ ]:
!pip install -q seaborn

In [ ]:
import tensorflow as tf
import pandas as pd
import matplotlib
from matplotlib import pyplot as plt
import seaborn as sns
import sklearn.metrics as sk_metrics
import tempfile
import os

# Preset matplotlib figure sizes.
matplotlib.rcParams['figure.figsize'] = [9, 6]

print(tf.__version__)
# To make the results reproducible, set the random seed value.
tf.random.set_seed(22)

## Carregando os dados

Em seguida, carregue o dataset [Wisconsin Breast Cancer](https://archive.ics.uci.edu/ml/datasets/breast+cancer+wisconsin+(original)){:.external} do [UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/){:.external}. Este dataset contém várias características, como raio, textura e concavidade de um tumor.

In [ ]:
url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wdbc.data'

features = ['radius', 'texture', 'perimeter', 'area', 'smoothness', 'compactness',
            'concavity', 'concave_poinits', 'symmetry', 'fractal_dimension']
column_names = ['id', 'diagnosis']

for attr in ['mean', 'ste', 'largest']:
  for feature in features:
    column_names.append(feature + "_" + attr)

Leia o dataset para um [DataFrame]() do pandas{:.external} usando [`pandas.read_csv`](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html){:.external}:

In [ ]:
dataset = pd.read_csv(url, names=column_names)

In [ ]:
dataset.info()

Exiba as primeiras cinco linhas:

In [ ]:
dataset.head()

Divida o dataset em conjuntos de treinamento e teste usando [`pandas.DataFrame.sample`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.sample.html){:.external}, [`pandas.DataFrame.drop`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html){:.external} e [`pandas.DataFrame.iloc`](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.iloc.html){:.external}. Não esqueça de dividir as características dos rótulos de destino. O dataset de testes é usado para avaliar a generalização do seu modelo para dados não vistos.

In [ ]:
train_dataset = dataset.sample(frac=0.75, random_state=1)

In [ ]:
len(train_dataset)

In [ ]:
test_dataset = dataset.drop(train_dataset.index)

In [ ]:
len(test_dataset)

In [ ]:
# The `id` column can be dropped since each row is unique
x_train, y_train = train_dataset.iloc[:, 2:], train_dataset.iloc[:, 1]
x_test, y_test = test_dataset.iloc[:, 2:], test_dataset.iloc[:, 1]

## Pré-processamento dos dados

Este dataset contém a média, o erro padrão e os maiores valores para cada uma das 10 medições de tumor coletadas em cada exemplo. A coluna alvo `"diagnosis"` é uma variável categórica onde `'M'` indica um diagnóstico de tumor maligno e `'B'`, um diagnóstico de tumor benigno. Esta coluna precisa ser convertida em formato binário numérico para treinamento do modelo.

A função [`pandas.Series.map`](https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html){:.external} é útil para mapear valores binários para as categorias.

O dataset também deve ser convertido para um tensor com a função `tf.convert_to_tensor` depois de concluído o pré-processamento.

In [ ]:
y_train, y_test = y_train.map({'B': 0, 'M': 1}), y_test.map({'B': 0, 'M': 1})
x_train, y_train = tf.convert_to_tensor(x_train, dtype=tf.float32), tf.convert_to_tensor(y_train, dtype=tf.float32)
x_test, y_test = tf.convert_to_tensor(x_test, dtype=tf.float32), tf.convert_to_tensor(y_test, dtype=tf.float32)

Use [`seaborn.pairplot`](https://seaborn.pydata.org/generated/seaborn.pairplot.html){:.external} para revisar a distribuição conjunta de alguns pares de recursos baseados em média do conjunto de treinamento e observar como eles se relacionam com o alvo:

In [ ]:
sns.pairplot(train_dataset.iloc[:, 1:6], hue = 'diagnosis', diag_kind='kde');

Este pairplot demonstra que certas características como raio, perímetro e área são altamente correlacionadas. Isto é esperado, uma vez que o raio do tumor está diretamente envolvido no cálculo do perímetro e da área. Além disso, observe que os diagnósticos malignos parecem ter um maior desvio para a direita em várias características.

Não deixe de verificar as estatísticas gerais. Observe como cada característica cobre uma faixa de valores muito diferente.

In [ ]:
train_dataset.describe().transpose()[:10]

Dados os intervalos inconsistentes, é vantajoso padronizar os dados de forma que cada característica tenha média zero e variância unitária. Esse processo é chamado de [normalização](https://developers.google.com/machine-learning/glossary#normalization){:.external}.

In [ ]:
class Normalize(tf.Module):
  def __init__(self, x):
    # Initialize the mean and standard deviation for normalization
    self.mean = tf.Variable(tf.math.reduce_mean(x, axis=0))
    self.std = tf.Variable(tf.math.reduce_std(x, axis=0))

  def norm(self, x):
    # Normalize the input
    return (x - self.mean)/self.std

  def unnorm(self, x):
    # Unnormalize the input
    return (x * self.std) + self.mean

norm_x = Normalize(x_train)
x_train_norm, x_test_norm = norm_x.norm(x_train), norm_x.norm(x_test)

## Regressão logística

Antes de construir um modelo de regressão logística, é fundamental compreender as diferenças do método em relação à regressão linear tradicional.

### Fundamentos da regressão logística

A regressão linear retorna uma combinação linear de suas entradas; esta saída é ilimitada. A saída de uma [regressão logística](https://developers.google.com/machine-learning/glossary#logistic_regression){:.external} está no intervalo `(0, 1)`. Para cada exemplo, ela representa a probabilidade de o exemplo pertencer à classe *positiva*.

A regressão logística mapeia os resultados contínuos da regressão linear tradicional, `(-∞, ∞)`, a probabilidades, `(0, 1)`. Essa transformação também é simétrica, de modo que a inversão do sinal da saída linear resulta no inverso da probabilidade original.

Seja $Y$ a probabilidade de estar na classe `1` (o tumor é maligno). O mapeamento desejado pode ser alcançado interpretando a saída da regressão linear como o [logaritmo da razão de probabilidades](https://developers.google.com/machine-learning/glossary#log-odds){:.external} de estar na classe `1` em vez de estar na classe `0`:

$$\ln(\frac{Y}{1-Y}) = wX + b$$

Ao definir $wX + b = z$, esta equação pode então ser resolvida para $Y$:

$$Y = \frac{e^{z}}{1 + e^{z}} = \frac{1}{1 + e^{-z}}$$

A expressão $\frac{1}{1 + e^{-z}}$ é conhecida como [função sigmóide](https://developers.google.com/machine-learning/glossary#sigmoid_function){:.external} $\sigma(z)$. Portanto, a equação para regressão logística pode ser escrita como $Y = \sigma(wX + b)$.

O dataset neste tutorial trata de uma matriz de características de alta dimensão. Portanto, a equação acima deve ser reescrita em forma vetorial matricial da seguinte forma:

$${\mathrm{Y}} = \sigma({\mathrm{X}}w + b)$$

onde:

- $\underset{m\times 1}{\mathrm{Y}}$: vetor alvo
- $\underset{m\times n}{\mathrm{X}}$: matriz de características
- $\underset{n\times 1}w$: vetor de peso
- $b$: bias
- $\sigma$: função sigmóide aplicada a cada elemento do vetor de saída

Comece visualizando a função sigmóide, que transforma a saída linear, `(-∞, ∞)`, para ficar entre `0` e `1`. A função sigmóide está disponível em `tf.math.sigmoid`.

In [ ]:
x = tf.linspace(-10, 10, 500)
x = tf.cast(x, tf.float32)
f = lambda x : (1/20)*x + 0.6
plt.plot(x, tf.math.sigmoid(x))
plt.ylim((-0.1,1.1))
plt.title("Sigmoid function");

### A função de perda logarítmica

A função de [perda logarítmica](https://developers.google.com/machine-learning/glossary#Log_Loss){:.external}, ou perda de entropia cruzada binária, é a função de perda ideal para um problema de classificação binária com regressão logística. Para cada exemplo, a perda logarítmica quantifica a semelhança entre uma probabilidade prevista e o valor verdadeiro do exemplo. É determinada pela seguinte equação:

$$L = -\frac{1}{m}\sum_{i=1}^{m}y_i\cdot\log(\hat{y}_i) + (1- y_i)\cdot\log(1 - \hat{y}_i)$$

onde:

- $\hat{y}$: um vetor de probabilidades previstas
- $y$: um vetor de alvos verdadeiros

Você pode usar a função `tf.nn.sigmoid_cross_entropy_with_logits` para calcular a perda logarítmica. Esta função aplica automaticamente a ativação sigmóide à saída da regressão:

In [ ]:
def log_loss(y_pred, y):
  # Compute the log loss function
  ce = tf.nn.sigmoid_cross_entropy_with_logits(labels=y, logits=y_pred)
  return tf.reduce_mean(ce)

### A regra de atualização do método do gradiente descendente

As APIs do TensorFlow Core oferecem suporte à diferenciação automática com `tf.GradientTape`. Se você tem curiosidade quanto à matemática por trás das [atualizações de gradiente](https://developers.google.com/machine-learning/glossary#gradient_descent) de regressão logística{:.external}, aqui está uma breve explicação:

Na equação acima para a perda logarítmica, lembre-se que cada $\hat{y}_i$ pode ser reescrito em termos das entradas como $\sigma({\mathrm{X_i}}w + b)$.

O objetivo é encontrar um $w^ {em0}$ e $b^{/em0}$ que minimize a perda logarítmica:

$$L = -\frac{1}{m}\sum_{i=1}^{m}y_i\cdot\log(\sigma({\mathrm{X_i}}w + b)) + (1- y_i)\cdot\log(1 - \sigma({\mathrm{X_i}}w + b))$$

Tomando o gradiente $L$ em relação a $w$, você obtém o seguinte:

$$\frac{\partial L}{\partial w} = \frac{1}{m}(\sigma({\mathrm{X}}w + b) - y)X$$

Tomando o gradiente $L$ em relação a $b$, você obtém o seguinte:

$$\frac{\partial L}{\partial b} = \frac{1}{m}\sum_{i=1}^{m}\sigma({\mathrm{X_i}}w + b) - y_i$$

Agora, construa o modelo de regressão logística.

In [ ]:
class LogisticRegression(tf.Module):

  def __init__(self):
    self.built = False
    
  def __call__(self, x, train=True):
    # Initialize the model parameters on the first call
    if not self.built:
      # Randomly generate the weights and the bias term
      rand_w = tf.random.uniform(shape=[x.shape[-1], 1], seed=22)
      rand_b = tf.random.uniform(shape=[], seed=22)
      self.w = tf.Variable(rand_w)
      self.b = tf.Variable(rand_b)
      self.built = True
    # Compute the model output
    z = tf.add(tf.matmul(x, self.w), self.b)
    z = tf.squeeze(z, axis=1)
    if train:
      return z
    return tf.sigmoid(z)

Para validar, certifique-se de que o modelo não treinado produza valores no intervalo de `(0, 1)` para um pequeno subconjunto de dados de treinamento.

In [ ]:
log_reg = LogisticRegression()

In [ ]:
y_pred = log_reg(x_train_norm[:5], train=False)
y_pred.numpy()

Em seguida, escreva uma função de exatidão para calcular a proporção de classificações corretas durante o treinamento. Para recuperar as classificações das probabilidades previstas, defina um limite para o qual todas as probabilidades superiores ao limite pertencem à classe `1`. Este é um hiperparâmetro configurável que pode ser definido como `0.5` por padrão.

In [ ]:
def predict_class(y_pred, thresh=0.5):
  # Return a tensor with  `1` if `y_pred` > `0.5`, and `0` otherwise
  return tf.cast(y_pred > thresh, tf.float32)

def accuracy(y_pred, y):
  # Return the proportion of matches between `y_pred` and `y`
  y_pred = tf.math.sigmoid(y_pred)
  y_pred_class = predict_class(y_pred)
  check_equal = tf.cast(y_pred_class == y,tf.float32)
  acc_val = tf.reduce_mean(check_equal)
  return acc_val

### Treine o modelo

O uso de minilotes para treinamento garante eficiência de memória e convergência mais rápida. A API `tf.data.Dataset` possui funções úteis para lote e embaralhamento. A API permite criar pipelines de entrada complexos a partir de peças simples e reutilizáveis. 

In [ ]:
batch_size = 64
train_dataset = tf.data.Dataset.from_tensor_slices((x_train_norm, y_train))
train_dataset = train_dataset.shuffle(buffer_size=x_train.shape[0]).batch(batch_size)
test_dataset = tf.data.Dataset.from_tensor_slices((x_test_norm, y_test))
test_dataset = test_dataset.shuffle(buffer_size=x_test.shape[0]).batch(batch_size)

Agora escreva um loop de treinamento para o modelo de regressão logística. O loop utiliza a função de perda logarítmica e seus gradientes em relação à entrada para atualizar iterativamente os parâmetros do modelo.

In [ ]:
# Set training parameters
epochs = 200
learning_rate = 0.01
train_losses, test_losses = [], []
train_accs, test_accs = [], []

# Set up the training loop and begin training
for epoch in range(epochs):
  batch_losses_train, batch_accs_train = [], []
  batch_losses_test, batch_accs_test = [], []

  # Iterate over the training data
  for x_batch, y_batch in train_dataset:
    with tf.GradientTape() as tape:
      y_pred_batch = log_reg(x_batch)
      batch_loss = log_loss(y_pred_batch, y_batch)
    batch_acc = accuracy(y_pred_batch, y_batch)
    # Update the parameters with respect to the gradient calculations
    grads = tape.gradient(batch_loss, log_reg.variables)
    for g,v in zip(grads, log_reg.variables):
      v.assign_sub(learning_rate * g)
    # Keep track of batch-level training performance
    batch_losses_train.append(batch_loss)
    batch_accs_train.append(batch_acc)

  # Iterate over the testing data
  for x_batch, y_batch in test_dataset:
    y_pred_batch = log_reg(x_batch)
    batch_loss = log_loss(y_pred_batch, y_batch)
    batch_acc = accuracy(y_pred_batch, y_batch)
    # Keep track of batch-level testing performance
    batch_losses_test.append(batch_loss)
    batch_accs_test.append(batch_acc)

  # Keep track of epoch-level model performance
  train_loss, train_acc = tf.reduce_mean(batch_losses_train), tf.reduce_mean(batch_accs_train)
  test_loss, test_acc = tf.reduce_mean(batch_losses_test), tf.reduce_mean(batch_accs_test)
  train_losses.append(train_loss)
  train_accs.append(train_acc)
  test_losses.append(test_loss)
  test_accs.append(test_acc)
  if epoch % 20 == 0:
    print(f"Epoch: {epoch}, Training log loss: {train_loss:.3f}")

### Avaliação de desempenho

Observe as mudanças na perda e na exatidão do seu modelo ao longo do tempo. 

In [ ]:
plt.plot(range(epochs), train_losses, label = "Training loss")
plt.plot(range(epochs), test_losses, label = "Testing loss")
plt.xlabel("Epoch")
plt.ylabel("Log loss")
plt.legend()
plt.title("Log loss vs training iterations");

In [ ]:
plt.plot(range(epochs), train_accs, label = "Training accuracy")
plt.plot(range(epochs), test_accs, label = "Testing accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy (%)")
plt.legend()
plt.title("Accuracy vs training iterations");

In [ ]:
print(f"Final training log loss: {train_losses[-1]:.3f}")
print(f"Final testing log Loss: {test_losses[-1]:.3f}")

In [ ]:
print(f"Final training accuracy: {train_accs[-1]:.3f}")
print(f"Final testing accuracy: {test_accs[-1]:.3f}")

O modelo demonstra alta exatidão e baixa perda quando se trata de classificar tumores no dataset de treinamento e também generaliza bem para os dados de teste não vistos. Para dar um passo adiante, você pode explorar as taxas de erro que fornecem mais informações além da pontuação geral de exatidão. As duas taxas de erro mais populares para problemas de classificação binária são a taxa de falsos positivos (FPR) e a taxa de falsos negativos (FNR).

Para este problema, o FPR é a proporção de previsões de tumores malignos entre os tumores que são realmente benignos. Por outro lado, o FNR é a proporção de previsões de tumores benignos entre os tumores que são realmente malignos.

Compute uma matriz de confusão usando [`sklearn.metrics.confusion_matrix`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix){:.external}, que avalia a exatidão da classificação, e use matplotlib para exibir a matriz:

In [ ]:
def show_confusion_matrix(y, y_classes, typ):
  # Compute the confusion matrix and normalize it
  plt.figure(figsize=(10,10))
  confusion = sk_metrics.confusion_matrix(y.numpy(), y_classes.numpy())
  confusion_normalized = confusion / confusion.sum(axis=1, keepdims=True)
  axis_labels = range(2)
  ax = sns.heatmap(
      confusion_normalized, xticklabels=axis_labels, yticklabels=axis_labels,
      cmap='Blues', annot=True, fmt='.4f', square=True)
  plt.title(f"Confusion matrix: {typ}")
  plt.ylabel("True label")
  plt.xlabel("Predicted label")

y_pred_train, y_pred_test = log_reg(x_train_norm, train=False), log_reg(x_test_norm, train=False)
train_classes, test_classes = predict_class(y_pred_train), predict_class(y_pred_test)

In [ ]:
show_confusion_matrix(y_train, train_classes, 'Training')

In [ ]:
show_confusion_matrix(y_test, test_classes, 'Testing')

Observe as medições da taxa de erro e interprete seu significado no contexto deste exemplo. Em muitos estudos de testes médicos, como a detecção de câncer, ter uma alta taxa de falsos positivos para garantir uma baixa taxa de falsos negativos é perfeitamente aceitável e, de fato, encorajado, uma vez que o risco de deixar escapar um diagnóstico de tumor maligno (falso negativo) é muito pior do que classificar erroneamente um tumor benigno como maligno (falso positivo).

Para controlar o FPR e o FNR, tente alterar o hiperparâmetro de limite (threshold) antes de classificar as previsões de probabilidade. Um limite mais baixo aumenta as chances gerais do modelo de fazer uma classificação de tumor maligno. Isto inevitavelmente aumenta o número de falsos positivos e o FPR, mas também ajuda a diminuir o número de falsos negativos e o FNR.

## Salve o modelo

Comece criando um módulo de exportação que receba dados brutos e execute as seguintes operações:

- Normalização
- Previsão de probabilidade
- Previsão de classe


In [ ]:
class ExportModule(tf.Module):
  def __init__(self, model, norm_x, class_pred):
    # Initialize pre- and post-processing functions
    self.model = model
    self.norm_x = norm_x
    self.class_pred = class_pred

  @tf.function(input_signature=[tf.TensorSpec(shape=[None, None], dtype=tf.float32)])
  def __call__(self, x):
    # Run the `ExportModule` for new data points
    x = self.norm_x.norm(x)
    y = self.model(x, train=False)
    y = self.class_pred(y)
    return y 

In [ ]:
log_reg_export = ExportModule(model=log_reg,
                              norm_x=norm_x,
                              class_pred=predict_class)

Você pode salvar o modelo em seu estado atual com a função `tf.saved_model.save`. Para carregar um modelo salvo e fazer previsões, use a função `tf.saved_model.load`.

In [ ]:
models = tempfile.mkdtemp()
save_path = os.path.join(models, 'log_reg_export')
tf.saved_model.save(log_reg_export, save_path)

In [ ]:
log_reg_loaded = tf.saved_model.load(save_path)
test_preds = log_reg_loaded(x_test)
test_preds[:10].numpy()

## Conclusão

Este caderno introduziu algumas técnicas para lidar com um problema de regressão logística. Aqui estão mais algumas dicas que podem ajudar:

- As [APIs do TensorFlow Core](https://www.tensorflow.org/guide/core) podem ser usadas para criar fluxos de trabalho de aprendizado de máquina com altos níveis de configurabilidade
- A análise das taxas de erro é uma ótima maneira de obter mais informações sobre o desempenho de um modelo de classificação além da pontuação geral de precisão.
- O overfitting é outro problema comum para modelos de regressão logística, embora não tenha sido um problema neste tutorial. Veja o tutorial [Overfit e underfit](../../tutorials/keras/overfit_and_underfit.ipynb) para obter informações sobre isto.

Para obter mais exemplos de uso das APIs Core do TensorFlow, confira o [guia](https://www.tensorflow.org/guide/core) . Se você quiser saber mais sobre como carregar e preparar dados, consulte os tutoriais sobre [carregamento de dados de imagem](../../tutorials/load_data/images.ipynb) ou [carregamento de dados CSV](../../tutorials/load_data/csv.ipynb).